In [12]:
import numpy as np
import matplotlib.pyplot as plt
import sigmf
from sigmf import sigmffile, SigMFFile

## Load the dataset

In [13]:
filename = 'data/dataset'
sigFile = sigmffile.fromfile(filename,skip_checksum=True)
annotations = sigFile.get_annotations()
nSignals = len(annotations)
# TODO: For now, I'm assuming all the data has the same length
nSamps = int(sigFile.sample_count/nSignals)
# The input data is a tensor with shape nSignals x 2 x nSamps
signals = np.zeros((nSignals,2,nSamps))
for iSignal,annotation in enumerate(annotations):
  # Start index of the current signal
  annotationStart = annotation[SigMFFile.START_INDEX_KEY]
  # Length of the current signal in samples
  annotationLength = annotation[SigMFFile.LENGTH_INDEX_KEY]
  # Data for the current signal
  samples = sigFile.read_samples(annotationStart,annotationLength)
  # Store the signal data in the input tensor
  # Note: Since the data is complex, we need to split it into real and imaginary
  # parts because neural networks have trouble handling complex data
  signals[iSignal,0,:] = np.real(samples)
  signals[iSignal,1,:] = np.imag(samples)
  


(2000, 2, 128)
(2000, 2, 128)
